# Scraping basics for Playwright

This notebook is a combination of small scraping techniques along with how to use Playwright. Along with the class notes, the [scraping section](https://jonathansoma.com/everything/scraping/) on my Everything I Know site might be helpful.

## Imports

Import what you need to use Playwright, and start up a new browser to use for scraping. 

> If you end up opening a lot of Chromes/Chromiums, shutting down the Python kernel with the stop button is an easy way to make them go away! You'll have to re-run your notebook, but at least you won't have sixty icons in your dock.

In [2]:
from playwright.async_api import async_playwright

In [3]:
playwright = await async_playwright().start()
browser = await playwright.chromium.launch(headless=False)

In [6]:
page = await browser.new_page()

In [8]:
await page.goto("http://jonathansoma.com/columbia/interactive-scrape/by-class.html")

<Response url='https://jonathansoma.com/columbia/interactive-scrape/by-class.html' request=<Request url='https://jonathansoma.com/columbia/interactive-scrape/by-class.html' method='GET'>>

## Scraping by class

Scrape the content at http://jonathansoma.com/columbia/interactive-scrape/by-class.html using their **class name**, printing out the title, subhead, and byline.

In [9]:
html = await page.content()
html

'<!DOCTYPE html><html><head><script>\n    const html = `\n<h1 class="title">How to Scrape Things</h1>\n<h3 class="subhead">Probably using Playwright</h3>\n<p class="byline">By Jonathan Soma</p>\n`\n\nsetTimeout(() => {\n    console.log(html)\n    document.querySelector(\'body\').innerHTML = html\n}, 250)</script>\n</head><body>\n<h1 class="title">How to Scrape Things</h1>\n<h3 class="subhead">Probably using Playwright</h3>\n<p class="byline">By Jonathan Soma</p>\n</body></html>'

In [13]:
from bs4 import BeautifulSoup
soup_doc = BeautifulSoup(html)
soup_doc

<!DOCTYPE html>
<html><head><script>
    const html = `
<h1 class="title">How to Scrape Things</h1>
<h3 class="subhead">Probably using Playwright</h3>
<p class="byline">By Jonathan Soma</p>
`

setTimeout(() => {
    console.log(html)
    document.querySelector('body').innerHTML = html
}, 250)</script>
</head><body>
<h1 class="title">How to Scrape Things</h1>
<h3 class="subhead">Probably using Playwright</h3>
<p class="byline">By Jonathan Soma</p>
</body></html>

In [19]:
title = soup_doc.find(class_="title").text
subhead = soup_doc.find(class_="subhead").text
byline= soup_doc.find(class_="byline").text
print (title, subhead, byline)

How to Scrape Things Probably using Playwright By Jonathan Soma


## Scraping using a single tag

Scrape the content at http://jonathansoma.com/columbia/interactive-scrape/by-list.html, creating a dictionary out of the title, subhead, and byline.

In [22]:
page = await browser.new_page()
await page.goto("http://jonathansoma.com/columbia/interactive-scrape/by-list.html")

<Response url='https://jonathansoma.com/columbia/interactive-scrape/by-list.html' request=<Request url='https://jonathansoma.com/columbia/interactive-scrape/by-list.html' method='GET'>>

In [24]:
html = await page.content()
html

"<!DOCTYPE html><html><head><script>\n    const html = `<p>How to Scrape Things</p>\n<p>Probably using Playwright</p>\n<p>By Jonathan Soma</p>\n`\n\nsetTimeout(() => {\n    console.log(html)\n    document.querySelector('body').innerHTML = html\n}, 250)</script>\n</head><body><p>How to Scrape Things</p>\n<p>Probably using Playwright</p>\n<p>By Jonathan Soma</p>\n</body></html>"

In [33]:
from bs4 import BeautifulSoup
soup_doc = BeautifulSoup(html)
soup_doc

<!DOCTYPE html>
<html><head><script>
    const html = `<p>How to Scrape Things</p>
<p>Probably using Playwright</p>
<p>By Jonathan Soma</p>
`

setTimeout(() => {
    console.log(html)
    document.querySelector('body').innerHTML = html
}, 250)</script>
</head><body><p>How to Scrape Things</p>
<p>Probably using Playwright</p>
<p>By Jonathan Soma</p>
</body></html>

In [35]:
soup_doc.find_all('p')[0]

<p>How to Scrape Things</p>

In [54]:
code = soup_doc.find('p')
code

<p>How to Scrape Things</p>

In [67]:
subtitle = soup_doc.find_all('p')[1]
subtitle

<p>Probably using Playwright</p>

In [72]:
code = soup_doc.find('p')
soma = []
for line in code:
    title = soup_doc.find_all('p')[0].text
    subtitle = soup_doc.find_all('p')[1].text
    byline = soup_doc.find_all('p')[2].text
    
    soma.append(title)
    soma.append(subtitle)
    soma.append(byline)

soma

['How to Scrape Things', 'Probably using Playwright', 'By Jonathan Soma']

## Waiting

Scrape the content at http://jonathansoma.com/columbia/interactive-scrape/by-tag-wait.html just like you above, but use  **wait_for** to wait for the text "Everything has shown up" to show up.

In [80]:
page = await browser.new_page()
await page.goto("http://jonathansoma.com/columbia/interactive-scrape/by-tag-wait.html")
await page.get_by_text("Everything has shown up").wait_for()

In [81]:
html = await page.content()
html

'<!DOCTYPE html><html><head><script>\n    const html = `<p>How to Scrape Things</p>\n<p>Probably using Playwright</p>\n<p>By Jonathan Soma</p>\n<p>Everything has shown up</p> \n`\n\nlet pieces = html.split("\\n")\n\nfunction addPiece() {\n    document.querySelector(\'body\').innerHTML = document.querySelector(\'body\').innerHTML + pieces.shift()\n    if(pieces.length > 0) {\n        setTimeout(addPiece, 250)\n    } else {\n        setTimeout(() => {\n            document.querySelector(\'body\').innerHTML = ""\n            pieces = html.split("\\n")\n            setTimeout(addPiece, 1000)\n        }, 2000)\n    }\n}\n\nsetTimeout(addPiece, 250)\n</script>\n</head><body></body></html>'

In [82]:
soup_doc = BeautifulSoup(html)
soup_doc

<!DOCTYPE html>
<html><head><script>
    const html = `<p>How to Scrape Things</p>
<p>Probably using Playwright</p>
<p>By Jonathan Soma</p>
<p>Everything has shown up</p> 
`

let pieces = html.split("\n")

function addPiece() {
    document.querySelector('body').innerHTML = document.querySelector('body').innerHTML + pieces.shift()
    if(pieces.length > 0) {
        setTimeout(addPiece, 250)
    } else {
        setTimeout(() => {
            document.querySelector('body').innerHTML = ""
            pieces = html.split("\n")
            setTimeout(addPiece, 1000)
        }, 2000)
    }
}

setTimeout(addPiece, 250)
</script>
</head><body></body></html>

## Forms

Display the content of the `h1` tag on http://jonathansoma.com/columbia/interactive-scrape/inputs.html. You'll need to follow the instructions to complete the form first.

In [101]:
page = await browser.new_page()
await page.goto("http://jonathansoma.com/columbia/interactive-scrape/inputs.html")
page = await page.content()
page
soup_doc = BeautifulSoup(html)
soup_doc

<!DOCTYPE html>
<html><head><script>
    const html = `<h1>You did it</h1>`
</script>
</head><body>
<div id="things"></div>
<p>The secret is
    <select>
<option selected="">Closed</option>
<option>Open</option>
</select>
</p>
<p>
<input id="best-animal" placeholder="write cat in here" type="text"/>
</p>
<p>
<input id="submit" type="button" value="Click me"/>
</p>
<script>
        document.querySelector("#submit").addEventListener('click', function() {
            if(document.querySelector('#best-animal').value == 'cat') {
                if(document.querySelector("select").value == 'Open') {
                    document.querySelector('body').innerHTML = html
                } else {
                    alert('fix the dropdown!!!')
                }
            } else {
                alert('write cat in there!!!')
            }
        })
    </script>
</body></html>

In [105]:
page.select_option("select", label="Open")

AttributeError: 'str' object has no attribute 'select_option'

In [92]:
await page.get_by_role("button", name="Click me").click()

AttributeError: 'str' object has no attribute 'get_by_role'

## Scraping a single table row

Scrape the content at http://jonathansoma.com/columbia/interactive-scrape/single-table-row.html, creating a dictionary out of the title, subhead, and byline.

## Saving into a dictionary

Scrape the content at http://jonathansoma.com/columbia/interactive-scrape/single-table-row.html, saving the title, subhead, and byline into a single dictionary called `book`.

> Don't use pandas for this one!

## Scraping multiple table rows

Scrape the content at http://jonathansoma.com/columbia/interactive-scrape/multiple-table-rows.html, creating a list of dictionaries. Convert to a pandas dataframe with `pd.json_normalize`. Save it as `output.csv`.

## Scraping an actual table

Scrape the content at http://jonathansoma.com/columbia/interactive-scrape/the-actual-table.html using pandas' HTML reading function. Save it as `output.csv`.

## `html.parser` vs `html5lib`

Here is some good HTML:

```python
html_good = """
<h1>This is a title</h1>
<h2>This is a subhead</h2>
<p>This is a paragraph</p>
<p>This is another paragraph</p>
"""

Here is some bad HTML:
    
html_bad = """
<h1>This is a title
<h2>This is a subhead
<p>This is a paragraph
<p>This is another paragraph
"""
```

When you're using BeautifulSoup, you can use different parsers, including `html.parser`, `html5lib` and `lxml`. Try both the good HTML and bad HTML with each parser and use `print(soup_doc.prettify())` to view the difference.

What is different about each one?

> You'll need to `pip install` for both html5lib and lxml. Since you aren't important them, they're coming from BeautifulSoup, you'll need to do **Kernel > Restart** and run from the top after installing to have them work.